## 3. Keyword/Text (FAQ) search with Korean Analyzer

### Analyzer

- https://learn.microsoft.com/en-us/azure/search/search-analyzers
- https://learn.microsoft.com/en-us/azure/search/index-add-language-analyzers

In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [2]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
)

In [3]:
import os
service_endpoint = os.getenv("AZSCH_ENDPOINT")  
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

print(service_endpoint)

https://iksch.search.windows.net


In [4]:
# Create a search index
def create_search_index(index_name):
    index_client = SearchIndexClient(
        endpoint=service_endpoint, credential=credential)
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="title", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="body", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="category", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True,
                        facetable=True, filterable=True),
        SearchableField(name="type", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True,
                        facetable=True, filterable=True),
        SearchableField(name="docid", type=SearchFieldDataType.String)
    ]

    # Create the search index
    index = SearchIndex(name=index_name, fields=fields)
    result = index_client.create_or_update_index(index)
    print(f' {result.name} created')

In [5]:
index_name = "ncfaqs-kor-index"
create_search_index(index_name)

 ncfaqs-kor-index created


In [6]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

In [7]:
import pandas as pd
from tqdm import tqdm

In [8]:
df = pd.read_json("./sample_faqs.json")
df.head()

,id,category,title,body
0,469829,계정,계정 인증을 해제하고 싶어요.,계정 인증 해제는 아래의 방법을 참고해 주시길 바랍니다.\n■ 계정 인증 해제 방법...
1,412428,게임,밤까마귀 장비를 강화했는데 이벤트 및 업적 카운트에 반영되지 않아요.,"강화를 시도해도 파괴되지 않는 '보호의 밤까마귀 강화 주문서'로 강화했을 경우, 이..."
2,398796,결제(PC),"웹 상점 - 결제 취소는 되었지만, 결제 금액이 환불되지 않았어요.",결제 수단인 카드사 혹은 결제사의 정책에 따라 최종 결제 환불까지 영업일 기준 1~...
3,398795,결제(PC),웹 상점 - 결제 수단 별 한도가 궁금해요.,결제 수단 별 한도는 각 카드사 및 결제사의 정책에 따라 변동될 수 있습니다.\n▶...
4,398794,결제(PC),웹 상점 - 상품을 구매했지만 보관함에 들어오지 않았어요.,"웹 상점에서 상품을 구매하면, 공식 사이트 내 대표 캐릭터로 설정된 캐릭터에게 지급..."


In [9]:
count = 0
batch_size = 40
for i in tqdm(range(0, len(df), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(df))
    
    documents = df[i:i_end].apply(
        lambda row: {'id': str(row.name), 
                     'title': row['title'], 
                     'body': row['body'],
                     'category': row['category'],
                     'type': "faq",
                     "docid": str(row["id"])
                    }, axis=1).to_list()
    
    result = search_client.upload_documents(documents)  

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:01<00:00,  1.66it/s]


In [10]:
from azure.search.documents.models import (
    QueryType,
    SearchMode
)

import json

def azsch_text_query(query, type=QueryType.Simple, mode=SearchMode.Any):

    results = search_client.search(  
        search_text=query,
        query_type=type,
        search_mode=mode,
        search_fields=["body", "title"],
        select=["docid", "body", "title", "category", "type"],
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 
    
    print("Search Results:")
    for i, result in enumerate(results, 1): 
        print(f"{i}) {result['@search.score']:.10f}: {result['docid']}, {result['title']}, {result['category']}")  


## Demo

- basic keyword search
- filter
- paging
- faceting

### basic search

In [11]:
results = azsch_text_query('해외 접속')

Search Results:
1) 4.3750230000: 8, 나이트크로우를 해외에서 플레이할 수 있나요?, 설치/실행(MO)
2) 4.3445706000: 118, 게임을 탈퇴하고 싶어요., 계정
3) 4.1424384000: 63, 결제 내역(영수증)은 어디서 확인하나요?, 결제(MO)
4) 3.1623788000: 24, 공식 커뮤니티의 대표 캐릭터를 변경하고 싶습니다., 커뮤니티
5) 3.0660737000: 141, 스토어에서 업데이트하라고 하지만 스토어에서는 업데이트되지 않아요., 설치/실행(MO)
6) 2.4545364000: 20, 상품 결제 중 오류가 발생했어요., 결제(MO)
7) 2.0736270000: 14, 네트워크 오류로 이용 중 접속이 끊기는 현상이 발생합니다., 시스템(MO)
8) 1.9787881000: 95, 이용 중인 계정이 제재되었어요., 계정
9) 1.9602762000: 142, 나이트크로우 이용 중 강제 종료, 끊김, 지연 현상으로 이용이 어렵습니다., 설치/실행(MO)
10) 1.8250712000: 59, 결제 내역(영수증)은 어디서 확인하나요?, 결제(PC)


In [12]:
results = azsch_text_query('해외 플레이')

Search Results:
1) 8.0072900000: 8, 나이트크로우를 해외에서 플레이할 수 있나요?, 설치/실행(MO)
2) 3.5803380000: 63, 결제 내역(영수증)은 어디서 확인하나요?, 결제(MO)
3) 3.5048090000: 22, PC버전에서 결제를 진행할 수는 없나요?, 결제(PC)
4) 3.2277384000: 131, DirectX 11 버전과 12버전이 무슨 차이가 있나요?, 설치/실행(PC)
5) 3.0171359000: 125, PC 권장 사양을 알고 싶어요., 설치/실행(PC)
6) 2.4545364000: 20, 상품 결제 중 오류가 발생했어요., 결제(MO)
7) 2.2651446000: 137, 나이트크로우를 이용하기 위한 권장 사양을 알고 싶어요., 설치/실행(MO)
8) 2.1015399000: 13, 전체화면에서 창모드로 변경하고 싶어요., 시스템(PC)
9) 2.0327604000: 140, 나이트크로우에 접속할 수 없어요., 설치/실행(MO)
10) 1.7512813000: 59, 결제 내역(영수증)은 어디서 확인하나요?, 결제(PC)


### Hightlight

In [13]:
def azsch_text_query_highlight(query):

    results = search_client.search(  
        search_text=query,
        query_type=QueryType.Simple,
        search_mode=SearchMode.Any,
        search_fields=["body", "title"],
        select=["docid", "body", "title", "category", "type"],
        highlight_fields="body",
        query_language="ko-kr",
        top=5 # for limiting text search
    ) 
    
    print("Search Results:")
    for i, result in enumerate(results, 1):  
        if (result["@search.highlights"]):
            print(f"{result['docid']}: {result['@search.score']:.10f}: {result['title']}")
            for highlight in result['@search.highlights']['body']:
                print(f"    {highlight}")
        else:
            print(f"{result['docid']}: {result['@search.score']:.10f}: {result['title']} - {result['body']}")  

In [ ]:
azsch_text_query_highlight("해외 접속")

Search Results:
8: 4.3750230000: 나이트크로우를 해외에서 플레이할 수 있나요?
    그러므로 <em>해외에서</em> 다운로드 및 플레이는 어려울 수 있는 점 양해 부탁드립니다.
118: 4.3445706000: 게임을 탈퇴하고 싶어요.
    나이트크로우의 계정 탈퇴를 위해서는 먼저 게임 내 <em>접속이</em> 이루어져야 합니다.<em>접속</em> 후 [메뉴] &gt; [설정] &gt; [계정] 순으로 이동한 뒤, 최하단에 위치한 [게임탈퇴] 버튼을 눌러 진행하실 수 있습니다.
※ 게임 탈퇴는 72시간의 대기 시간이 적용됩니다. 72시간이 지나지 않은 상태에서 탈퇴 신청한 계정으로 재<em>접속</em>할 시에는 [게임 탈퇴 철회] 팝업창이 노출됩니다.※ 게임 탈퇴 대기 시간이 지난 후에는 게임 내 모든 정보가 삭제되며 복구가 불가능하오니 신중히 선택하시길 바랍니다.
63: 4.1424384000: 결제 내역(영수증)은 어디서 확인하나요?
    구글 플레이 스토어에 연결된 구글 계정으로 Gmail에 <em>접속</em>합니다.2.
    구글 페이먼트(https://pay.google.com) 사이트에 <em>접속</em>합니다.2.
24: 3.1623788000: 공식 커뮤니티의 대표 캐릭터를 변경하고 싶습니다.
     
■ 나이트 크로우 공식 커뮤니티 대표 캐릭터 변경 방법나이트 크로우 공식 커뮤니티 <em>접속</em> &gt; 캐릭터가 속해있는 연동 소셜 계정으로 로그인 &gt; 화면 우측 중단 직업 아이콘 클릭 &gt; 대표 캐릭터 변경 클릭 &gt; 대표 캐릭터로 설정할 캐릭터명 선택 &gt; 확인 버튼 클릭
141: 3.0660737000: 스토어에서 업데이트하라고 하지만 스토어에서는 업데이트되지 않아요.
    나이트크로우의 게임 <em>접속이</em> 원활치 않다면 아래 내용을 확인해주시길 바랍니다.
1.
    ＊다수의 사람들이 모여있는 장소 혹은 밀폐된 공간에서는 네트워크 <em>접속이</em> 원활하지

In [ ]:
azsch_text_query_highlight("해외 플레이")

Search Results:
8: 8.0072900000: 나이트크로우를 해외에서 플레이할 수 있나요?
    그러므로 <em>해외에서</em> 다운로드 및 <em>플레이는</em> 어려울 수 있는 점 양해 부탁드립니다.
63: 3.5803380000: 결제 내역(영수증)은 어디서 확인하나요?
    구글 <em>플레이</em> 스토어에 연결된 구글 계정으로 Gmail에 접속합니다.2.
22: 3.5048090000: PC버전에서 결제를 진행할 수는 없나요?
    나이트 크로우에서는 구글 <em>플레이를</em> 통한 PC버전에서의 결제를 지원하고 있습니다.
    [PC 버전 결제 (구글<em>플레이</em>) 가이드 바로가기]
131: 3.2277384000: DirectX 11 버전과 12버전이 무슨 차이가 있나요?
    .- DirectX 11 버전 : 기본으로 제공하는 버전- DirectX 12 버전 : CPU, GPU의 효율을 높여 더욱 뛰어난 그래픽 품질을 제공하는 버전
DirectX 12 버전은 기본으로 제공하는 DirectX 11 버전보다 더욱 뛰어나고 안정적인 그래픽 품질로 게임을 <em>플레이</em>할 수 있습니다.
125: 3.0171359000: PC 권장 사양을 알고 싶어요.
    .※ 나이트크로우는 32bit 운영 체제의 PC에서는 <em>플레이가</em> 불가하며, 64bit의 운영 체제에서만 이용이 가능합니다.


In [18]:
azsch_text_query_highlight("상품 구매 실패")

Search Results:
64: 12.3807720000: 구매한 상품은 어디에서 받을 수 있나요?
    나이트 크로우 상점에서 <em>구매</em>한 <em>상품은</em> 상점 내 <em>상품</em> 보관함에서 확인하실 수 있습니다.
■ <em>상품</em> 수령 방법[메뉴] &gt; [상점] &gt; [<em>상품</em> 보관함]에서 <em>구매</em>하신 <em>상품을</em> 수령할 수 있습니다.
229992: 12.2555870000: 상품 구매에 실패하였습니다.(P:-477) 에러가 발생해요.
    <em>상품</em> <em>구매에</em> <em>실패</em>하였습니다.
21: 11.5182990000: 구매한 상품은 어디에서 받을 수 있나요?
    나이트 크로우 상점에서 <em>구매</em>한 <em>상품은</em> 상점 내 <em>상품</em> 보관함에서 확인하실 수 있습니다
     
■ <em>상품</em> 수령 방법[메뉴] &gt; [상점] &gt; [<em>상품</em> 보관함]에서 <em>구매</em>하신 <em>상품을</em> 수령할 수 있습니다.
398794: 9.8779660000: 웹 상점 - 상품을 구매했지만 보관함에 들어오지 않았어요.
    웹 상점에서 <em>상품을</em> <em>구매</em>하면, 공식 사이트 내 대표 캐릭터로 설정된 캐릭터에게 지급됩니다.＊ 지급된 <em>상품은</em> [메뉴 &gt; 우편함 &gt; 시스템]에서 수령할 수 있습니다.
60: 9.4236555000: 상품을 환불하고 싶어요.
    결제하신 <em>상품은</em> <em>구매일로부터</em> 7일 이내 상점 내 <em>상품</em> 보관함에서 수령하지 않은 <em>상품에</em> 한하여 청약 철회가 가능합니다.청약 철회를 위해서는 서버, 캐릭터명과 결제를 진행하신 스토어, <em>구매</em> 일시, 주문번호를 포함하여 문의 접수를 부탁드립니다.※ 결제한 <em>상품을</em> 이미 수령/사용했거나 결제 즉시 적용되

In [19]:
azsch_text_query_highlight("마력 확률")

Search Results:
38233: 8.7895240000: 마력 주입 확률이 궁금해요.
    <em>마력</em> 주입 <em>확률을</em> 안내해 드립니다.
    <em>마력</em> 주입<em>마력</em> 주입 I구분능력치수치<em>확률</em>1번 능력치PVE 추가 피해315.00%PVE 추가 피해210.00%PVE 추가 피해110.00%생명력 물약 회복량1015.00%생명력 물약 회복량85.00%생명력 물약 회복량65.00%생명력 물약 회복량45.00%생명력 물약 회복량24.50%생명력 물약 회복량10.50%생명력 회복량515.00%생명력 회복량43.75%생명력 회복량33.75%생명력 회복량23.75%생명력 회복량13.75%2번 능력치PVE 명중215.00%PVE 명중120.00%최대 생명력10015.00%최대 생명력872.14%최대 생명력742.14%최대 생명력612.14%최대 생명력482.14%최대 생명력352.14%최대 생명력222.15%최대 생명력102.15%생명력 물약 회복률50015.00%생명력 물약 회복률4005.00%생명력 물약 회복률3005.00%생명력 물약 회복률2005.00%생명력 물약 회복률1005.00%3번 능력치PVE 치명타315.00%PVE 치명타210.00%PVE 치명타110.00%최대 정신력6015.00%최대 정신력532.14%최대 정신력462.14%최대 정신력392.14%최대 정신력322.14%최대 정신력252.14%최대 정신력182.15%최대 정신력102.15%최대 무게5,00015.00%최대 무게4,0005.00%최대 무게3,0005.00%최대 무게2,0005.00%최대 무게1,0005.00%추가 능력치방어2100.00%<em>마력</em> 주입 II구분능력치수치<em>확률</em>1번 능력치기술 피해45.00%기술 피해35.00%기술 피해25.00%기술 피해15.00%정신력 회복량710.00%정신력 회복량63.33%정신력 회복량53.33%정신력 회복량43.33%정신력 회복량33.33%정신력 회복량23.

In [20]:
azsch_text_query_highlight("마력확율")

Search Results:
38233: 5.6241016000: 마력 주입 확률이 궁금해요.
    <em>마력</em> 주입 확률을 안내해 드립니다.
    <em>마력</em> 주입<em>마력</em> 주입 I구분능력치수치확률1번 능력치PVE 추가 피해315.00%PVE 추가 피해210.00%PVE 추가 피해110.00%생명력 물약 회복량1015.00%생명력 물약 회복량85.00%생명력 물약 회복량65.00%생명력 물약 회복량45.00%생명력 물약 회복량24.50%생명력 물약 회복량10.50%생명력 회복량515.00%생명력 회복량43.75%생명력 회복량33.75%생명력 회복량23.75%생명력 회복량13.75%2번 능력치PVE 명중215.00%PVE 명중120.00%최대 생명력10015.00%최대 생명력872.14%최대 생명력742.14%최대 생명력612.14%최대 생명력482.14%최대 생명력352.14%최대 생명력222.15%최대 생명력102.15%생명력 물약 회복률50015.00%생명력 물약 회복률4005.00%생명력 물약 회복률3005.00%생명력 물약 회복률2005.00%생명력 물약 회복률1005.00%3번 능력치PVE 치명타315.00%PVE 치명타210.00%PVE 치명타110.00%최대 정신력6015.00%최대 정신력532.14%최대 정신력462.14%최대 정신력392.14%최대 정신력322.14%최대 정신력252.14%최대 정신력182.15%최대 정신력102.15%최대 무게5,00015.00%최대 무게4,0005.00%최대 무게3,0005.00%최대 무게2,0005.00%최대 무게1,0005.00%추가 능력치방어2100.00%<em>마력</em> 주입 II구분능력치수치확률1번 능력치기술 피해45.00%기술 피해35.00%기술 피해25.00%기술 피해15.00%정신력 회복량710.00%정신력 회복량63.33%정신력 회복량53.33%정신력 회복량43.33%정신력 회복량33.33%정신력 회복량23.34%정신력 회복량13.34%최대 정신력7015.